In [1]:
import numpy as np
import pickle

In [2]:
# Load precomputed jensen_shannon distance matrix
with open('/data/SS_RNA_seq/Code/clustering_on_reads/Trapnell_SJ.dat', 'rb') as infile:
    D = pickle.load(infile)  
with open('/data/SS_RNA_seq/Code/clustering_on_reads/Trapnell_sparse_eq_class_unnormalised.dat', 'rb') as infile:
    Xu = pickle.load(infile)
num_reads=Xu.sum(axis=1)
cells_sorted_by_num_reads=sorted(range(len(num_reads)), key=lambda k: num_reads[k], reverse=True)

#exclude bulk and low coverage
Dcleaned=D[np.ix_(cells_sorted_by_num_reads[:282],cells_sorted_by_num_reads[:282])]    
Dc = Dcleaned[np.ix_(range(24,260),range(24,260))]
    
    
    
    
Ds = np.rint(Dc*100000)
Ds= Ds.astype(int)

In [3]:
A=Ds
zc=np.zeros((np.size(A[:,1]),1), dtype=np.int)
zr=np.zeros((1,np.size(A[:,1])+1), dtype=np.int)
A=np.c_[zc,A]
A=np.r_[zr,A]

In [4]:
f = open("TSP_instance_trapnell.tsp", "w")
#write header
f.writelines(["NAME : TSP instance trapnell"+'\n',
"COMMENT : equivalence class clustering"+'\n',
"TYPE : TSP\n",
"DIMENSION :" + str(np.size(A[:,1]))+'\n',
"EDGE_WEIGHT_TYPE : EXPLICIT"+'\n',
"EDGE_WEIGHT_FORMAT : FULL_MATRIX"+'\n',
"EDGE_WEIGHT_SECTION"+'\n'])
#write edge weights
for i in range(np.size(A[:,1])):
    row_str = ' '.join(['%d' % num for num in A[i,:]])+'\n'
    f.write(row_str)
f.write("EOF")   
f.close()

## Run Concorde to solve TSP_instance.tsp  .... 
## ... and then continue here to extract/save the optimal path:

In [5]:
#read the tsp_solution and get opt path
with open('/data/software/concorde/TSP/TSP_instance_trapnell.sol', 'r') as infile:
    lines=infile.readlines()
path=[]
for line in lines[1:]:
    path = path+ [int(s) for s in line[:-2].split(' ')]

#sanity check
dist=np.zeros(237)
for i in range(236):
        dist[i]=A[i,i+1]
print "the direct traversal has distance: " + str(dist.sum())
           
dist=np.zeros(237)
for i in range(236):
        dist[i]=A[path[i],path[i+1]]
print "the optimal traversal has distance: "+str(dist.sum())    
    
#remove dummy node and make it zero-indexed    
path=path[1:]  
path=path -np.ones(236)   

#check 
print path
path.max()




the direct traversal has distance: 12468556.0
the optimal traversal has distance: 11284017.0
[ 235.  147.    7.   13.  124.  140.    2.   22.  185.  202.  138.   35.
   26.  195.  165.   90.   52.  183.   95.  203.   81.   41.  121.   40.
  112.   83.  109.   51.  210.  120.  169.  159.   94.  224.  204.   98.
   56.  164.  219.  184.  179.   73.  223.   14.  175.  135.  123.   10.
   55.   70.   19.  131.   43.  231.  166.  162.   91.   78.   88.   58.
   30.  163.  116.  113.  201.   77.  160.   61.   32.   18.   71.   47.
  157.  117.   46.  213.   86.  178.  111.   59.  102.  198.  128.   39.
   64.  144.  119.   53.   23.  200.  217.  209.   25.  168.  180.  156.
  139.   15.  216.  205.  228.   68.   75.  132.  211.  199.  220.   84.
  206.   34.  212.  227.  137.   44.  153.   54.  118.   36.  125.  193.
  146.  214.   69.  181.   76.   82.  150.  136.  148.   89.  230.   97.
  187.  182.  100.   21.  170.   65.   45.   60.  161.   80.   24.   48.
   27.  115.  126.  108.   79. 

235.0

In [6]:
#Save path
pickle.dump( path, open( "TSP_sol_path_trapnell.dat", "wb" ) )